In [ ]:
from google.colab import drive
drive.mount('/content/drive')


FOLDERNAME = "Computer Vision Project/kits23"
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1tnt3cKRdWn9lFKrM82KlrZH6CO5EgchD/Computer Vision Project/kits23


In [ ]:
!pip install -q "monai-weekly[nibabel, tqdm]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 56.2 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import albumentations as A

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

from monai.losses import DiceLoss

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
def augment(image, mask):
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        A.ElasticTransform(alpha=1, sigma=50, p=0.5),
    ])
    augmented = transform(image=image, mask=mask)
    return augmented["image"], augmented["mask"]

In [ ]:
class CTDataset(Dataset):
    def __init__(self, dataframe, train, transform=None):
        self.data = dataframe
        self.train = train
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx]['image']
        label_path = self.data.iloc[idx]['label']

        image = torch.load(image_path)
        label = torch.load(label_path)

        if self.train:
           image = image.squeeze(0)
           image, label = augment(image.numpy(), label.numpy())
           image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)
           label = torch.tensor(label, dtype=torch.long)

        # label = F.one_hot(label, num_classes=4).permute(3, 0, 1, 2).float()

        if self.transform:
           image, label = self.transform(image, label)

        return image, label

In [ ]:
class UNet3D(nn.Module):
    def __init__(self):
        super(UNet3D, self).__init__()
        # Encoder
        self.enc1 = self.conv_block(1, 32)
        self.enc2 = self.conv_block(32, 64)
        self.enc3 = self.conv_block(64, 128)
        self.enc4 = self.conv_block(128, 256)
        self.enc5 = self.conv_block(256, 512)
        self.pool = nn.MaxPool3d(2)

        # Decoder
        self.up5 = nn.ConvTranspose3d(512, 256, kernel_size=2, stride=2)
        self.dec5 = self.conv_block(512, 256)
        self.up4 = nn.ConvTranspose3d(256, 128, kernel_size=2, stride=2)
        self.dec4 = self.conv_block(256, 128)
        self.up3 = nn.ConvTranspose3d(128, 64, kernel_size=2, stride=2)
        self.dec3 = self.conv_block(128, 64)
        self.up2 = nn.ConvTranspose3d(64, 32, kernel_size=2, stride=2)
        self.dec2 = self.conv_block(64, 32)
        self.dec1 = nn.Conv3d(32, 4, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))
        enc5 = self.enc5(self.pool(enc4))

        # Decoder with skip connections
        dec5 = self.up5(enc5)
        dec5 = torch.cat([dec5, enc4], dim=1)
        dec5 = self.dec5(dec5)

        dec4 = self.up4(dec5)
        dec4 = torch.cat([dec4, enc3], dim=1)
        dec4 = self.dec4(dec4)

        dec3 = self.up3(dec4)
        dec3 = torch.cat([dec3, enc2], dim=1)
        dec3 = self.dec3(dec3)

        dec2 = self.up2(dec3)
        dec2 = torch.cat([dec2, enc1], dim=1)
        dec2 = self.dec2(dec2)

        # Output layer
        dec1 = self.dec1(dec2)
        return dec1

In [ ]:
model = UNet3D()

model.to('cuda')
input = (1, 128, 128, 128)
summary(model, input)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1    [-1, 32, 128, 128, 128]             896
       BatchNorm3d-2    [-1, 32, 128, 128, 128]              64
              ReLU-3    [-1, 32, 128, 128, 128]               0
            Conv3d-4    [-1, 32, 128, 128, 128]          27,680
       BatchNorm3d-5    [-1, 32, 128, 128, 128]              64
              ReLU-6    [-1, 32, 128, 128, 128]               0
         MaxPool3d-7       [-1, 32, 64, 64, 64]               0
            Conv3d-8       [-1, 64, 64, 64, 64]          55,360
       BatchNorm3d-9       [-1, 64, 64, 64, 64]             128
             ReLU-10       [-1, 64, 64, 64, 64]               0
           Conv3d-11       [-1, 64, 64, 64, 64]         110,656
      BatchNorm3d-12       [-1, 64, 64, 64, 64]             128
             ReLU-13       [-1, 64, 64, 64, 64]               0
        MaxPool3d-14       [-1, 64, 32,

In [ ]:
class DC_and_CE_loss(nn.Module):
    def __init__(self, weight_ce=1.0, weight_dice=1.0):
        super(DC_and_CE_loss, self).__init__()
        self.weight_ce = weight_ce
        self.weight_dice = weight_dice
        self.dice_loss = DiceLoss()
        self.ce_loss = nn.CrossEntropyLoss()

    def forward(self, net_output: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        target_one_hot = F.one_hot(target, 4).permute(0, 4, 1, 2, 3).float()
        dice_loss = self.dice_loss(net_output, target_one_hot)
        ce_loss = self.ce_loss(net_output, target)
        total_loss = self.weight_ce * ce_loss + self.weight_dice * dice_loss
        return total_loss

In [ ]:
train_image = [os.path.join("../preprocessed_image_train_2nd", path) for path in os.listdir("../preprocessed_image_train_2nd")]
train_segment = [os.path.join("../preprocessed_segment_train_2nd", path) for path in os.listdir("../preprocessed_segment_train_2nd")]

print(train_image)
print(train_segment)
print(len(train_image))
print(len(train_segment))

['../preprocessed_image_train_2nd/image_case_00236_0.pth', '../preprocessed_image_train_2nd/image_case_00236_1.pth', '../preprocessed_image_train_2nd/image_case_00236_2.pth', '../preprocessed_image_train_2nd/image_case_00236_3.pth', '../preprocessed_image_train_2nd/image_case_00236_4.pth', '../preprocessed_image_train_2nd/image_case_00236_5.pth', '../preprocessed_image_train_2nd/image_case_00236_6.pth', '../preprocessed_image_train_2nd/image_case_00236_7.pth', '../preprocessed_image_train_2nd/image_case_00236_8.pth', '../preprocessed_image_train_2nd/image_case_00236_9.pth', '../preprocessed_image_train_2nd/image_case_00051_0.pth', '../preprocessed_image_train_2nd/image_case_00051_1.pth', '../preprocessed_image_train_2nd/image_case_00051_2.pth', '../preprocessed_image_train_2nd/image_case_00051_3.pth', '../preprocessed_image_train_2nd/image_case_00051_4.pth', '../preprocessed_image_train_2nd/image_case_00051_5.pth', '../preprocessed_image_train_2nd/image_case_00051_6.pth', '../preproces

In [ ]:
train_df = pd.DataFrame({'image': train_image, 'label': train_segment})

# Create datasets and dataloaders
dataset = CTDataset(train_df, train=True)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Define Model
model = UNet3D()
criterion = DC_and_CE_loss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
print('loaded model')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

start_epoch = 2
num_epochs = 5

# Load checkpoint
checkpoint_path = '/content/drive/MyDrive/Computer Vision Project/UNET_128_2nd/checkpoint_UNet_epoch_4.pth'
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Checkpoint loaded. Resuming training from epoch {start_epoch}.")


if not os.path.exists('../UNET_128_2nd'):
  os.makedirs('../UNET_128_2nd')

# Training loop
for epoch in tqdm(range(start_epoch, num_epochs + 1)):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(dataloader, leave=True):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        print('loss', loss.item())

        running_loss += loss.item()

    print(f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

    # Save checkpoint
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, f"../UNET_128_2nd/checkpoint_UNet_epoch_{epoch}.pth")
    print(f'Checkpoint saved at epoch {epoch}.')

loaded model


<ipython-input-9-b336c43a3433>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Checkpoint loaded. Resuming training from epoch 5.


  0%|          | 0/1560 [00:00<?, ?it/s]<ipython-input-4-2343cbc7ab94>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  image = torch.load(image_path)
<ipython-input-4-2343

loss 0.028366290032863617


...

loss 0.1438647210597992



100%|██████████| 1560/1560 [2:24:58<00:00,  5.58s/it]

loss 0.01014527678489685
Epoch [5/5], Loss: 0.1023



100%|██████████| 1/1 [2:25:01<00:00, 8701.95s/it]

Checkpoint saved at epoch 5.
